In [11]:
import pandas as pd
import numpy as np

In [2]:
! ls

data                  nlp-poetry-authorship pre-process.ipynb


In [13]:
cd ./data/train_val_test_split

/Users/christineshen/Desktop/NYU/Fall 2020/NLP/projects/data/train_val_test_split


In [13]:
! ls

test_data.csv  train_data.csv val_data.csv


In [14]:
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')
test_df = pd.read_csv('test_data.csv')

In [15]:
train_df.head()

,author,title,poetry_foundation_id,content,author_poem_count,author_poem_index,author_poem_pct
0,Percy sshe Shelley,from\n \n Queen Mab: Part VI,45137,"(excerpt)\n""Throughout these infinite orbs of ...",43,0,0.0
1,Thomas Hardy,'According to the Mighty Working',57342,I\n\nWhen moiling seems at cease\nIn the vague...,38,0,0.0
2,Rae Armantrout,Our Nature,54881,The very flatness\nof portraits\nmakes for nos...,62,0,0.0
3,Walt Whitman,For You O Democracy,51567,"Come, I will make the continent indissoluble,\...",41,0,0.0
4,William Butler Yeats,The Magi,12892,Now as at all times I can see in the mind's ey...,47,0,0.0


In [18]:
train_df.author.nunique()

39

In [21]:
train_df.content[1]

"I\n\nWhen moiling seems at cease\nIn the vague void of night-time,\nAnd heaven's wide roomage stormless\nBetween the dusk and light-time,\nAnd fear at last is formless,\nWe call the allurement Peace.\n\nII\n\nPeace, this hid riot, Change,\nThis revel of quick-cued mumming,\nThis never truly being,\nThis evermore becoming,\nThis spinner's wheel onfleeing\nOutside perception's range."

In [23]:
pip install transformers 

     |████████████████████████████████| 1.3 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 17.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [36]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')


In [19]:
exp = train_df.content[1]

In [20]:
exp

"I\n\nWhen moiling seems at cease\nIn the vague void of night-time,\nAnd heaven's wide roomage stormless\nBetween the dusk and light-time,\nAnd fear at last is formless,\nWe call the allurement Peace.\n\nII\n\nPeace, this hid riot, Change,\nThis revel of quick-cued mumming,\nThis never truly being,\nThis evermore becoming,\nThis spinner's wheel onfleeing\nOutside perception's range."

In [28]:
#check out the how the tokenizer work on the example 
tokenizer(exp)

{'input_ids': [101, 146, 1332, 182, 20708, 1158, 3093, 1120, 14092, 1130, 1103, 14673, 13340, 1104, 1480, 118, 1159, 117, 1262, 9775, 112, 188, 2043, 1395, 2553, 4162, 2008, 3847, 1103, 24494, 1105, 1609, 118, 1159, 117, 1262, 2945, 1120, 1314, 1110, 1532, 2008, 117, 1284, 1840, 1103, 1155, 3313, 1880, 5370, 119, 1563, 5370, 117, 1142, 11269, 14807, 117, 9091, 117, 1188, 1231, 12559, 1104, 3613, 118, 18780, 1181, 23993, 5031, 117, 1188, 1309, 5098, 1217, 117, 1188, 1518, 4982, 2479, 117, 1188, 6898, 2511, 112, 188, 4829, 1113, 23445, 20309, 1403, 9572, 11170, 112, 188, 2079, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [2]:
import torch
# import torch.nn as nn
# from torch import optim
# import torch.nn.functional as F
from torch.utils.data import DataLoader
# from torch.optim.lr_scheduler import ReduceLROnPlateau

import os
import time
from tqdm import notebook
from functools import partial

In [7]:
from torch.utils.data import Dataset

#ref: https://medium.com/analytics-vidhya/text-classification-with-bert-using-transformers-for-long-text-inputs-f54833994dfd

In [3]:
'''
def get_split(text1):
    l_total = []
    l_partial = []
    if len(text1.split())//150 > 0:
        n = len(text1.split())//150
    else:
        n = 1
    for w in range(n):
        if w == 0:
            l_partial = text1.split()[:200]
            l_total.append(" ".join(l_partial))
        else:
            l_partial = text1.split()[w*150:w*150 + 200]
            l_total.append(" ".join(l_partial))
    return l_total 
'''

In [8]:
class ReviewDataset(Dataset):
    def __init__(self, poems, targets, tokenizer, max_len):
        self.poems = poems
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len 
    
    def __len__(self):
        return len(self.poems)
    
    def __getitem__(self, item):
        poem = str(self.poems[item])
        target = self.targets[item]
        
        encoding = self.tokenizer.encode_plus(
            poem,
            add_special_tokens = True,
            max_length = self.max_len,
            return_token_type_ids = False,
            pad_to_max_length = True,
            return_attention_mask = True,
            return_tensors = 'pt',)
        
        output = {
            'poem_text': poem,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype = torch.long)
        }
        
        return output

In [38]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = ReviewDataset(
            poems = df.content.to_numpy(),
            targets = df.author.to_numpy(),
            tokenizer = tokenizer,
            max_len = max_len)
    
    return DataLoader(
        ds,
        batch_size = batch_size,
        shuffle = True
    )

In [25]:
train_df['source_len'] = train_df['content'].apply(lambda x: len(x.replace('\n',' ').split()))


In [33]:
MAX_LEN = int(train_df['source_len'].quantile(0.999)) ###
MAX_LEN

9632

In [34]:
batchSize = 64

In [39]:
train = create_data_loader(train_df, tokenizer, MAX_LEN, batchSize)

In [40]:
train